<a href="https://colab.research.google.com/github/andzagz/Logistic/blob/main/package_number.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pulp

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.7/17.7 MB 34.6 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np

# Generate data
np.random.seed(0)
cubagem_data = pd.DataFrame({
    'ID do Pacote': range(1, 101),
    'Peso': np.random.uniform(1, 50, 100),
    'Altura': np.random.uniform(10, 100, 100),
    'Largura': np.random.uniform(10, 100, 100),
    'Profundidade': np.random.uniform(10, 100, 100),
    'Tipo de Produto': np.random.choice(['Eletrônicos', 'Farmacêuticos', 'Vestuário'], 100),
    'Fragilidade': np.random.choice([True, False], 100),
    'ID do Veículo': np.random.choice(['V1', 'V2', 'V3'], 100),
    'Capacidade do Veículo': np.random.choice([200, 300, 400], 100)
})

print(cubagem_data['Capacidade do Veículo'])

0     400
1     400
2     300
3     200
4     200
     ... 
95    400
96    300
97    400
98    400
99    200
Name: Capacidade do Veículo, Length: 100, dtype: int64


In [ ]:
# Function to calculate the volume of each package (in m³)
cubagem_data['Volume (m³)'] = (cubagem_data['Altura'] * cubagem_data['Largura'] * cubagem_data['Profundidade']) / 1e6

# Sum the total weight (kg) and total volume (m³)
total_weight = cubagem_data['Peso'].sum()
total_volume = cubagem_data['Volume (m³)'].sum()

total_weight, total_volume


(2416.689813611337, 15.923983198391806)

In [ ]:
# Sum the weight and volume by vehicle
cubagem_data['Volume'] = cubagem_data['Altura'] * cubagem_data['Largura'] * cubagem_data['Profundidade'] / 1e6  # Converting to m³

# Group by vehicle and sum weight and volume
agrupado_veiculos = cubagem_data.groupby('ID do Veículo').agg(
    Total_weight=('Peso', 'sum'),
    Total_volume=('Volume', 'sum'),
    Total_packages=('ID do Pacote', 'count')
)

agrupado_veiculos


,Peso_total,Volume_total,Pacotes_total
ID do Veículo,,,
V1,777.972935,5.469259,37
V2,894.825951,5.820854,35
V3,743.890927,4.633871,28


In [ ]:
import numpy as np
import pandas as pd
from pulp import LpMaximize, LpProblem, LpVariable, lpSum, LpStatus

# Data
np.random.seed(0)
pacotes = list(range(1, 101))  # 100 packages with ID from 1 to 100
pesos = np.random.uniform(1, 50, 100)  # Weights between 1 and 50 kg

veiculos = ['V1', 'V2', 'V3']
capacidade_peso = {'V1': 200, 'V2': 300, 'V3': 400}

# Creating the maximization problem
prob = LpProblem("Maximize_Vehicle_Utilization", LpMaximize)

# Decision variables: x[i][j] = 1 if package i is allocated to vehicle j
x = LpVariable.dicts("x", [(i, j) for i in pacotes for j in veiculos], cat="Binary")

# Objective function: maximize the number of allocated packages
prob += lpSum(x[i, j] for i in pacotes for j in veiculos)

# Constraint 1: Total weight in each vehicle must be less than or equal to capacity
for j in veiculos:
    prob += lpSum(x[i, j] * pesos[i-1] for i in pacotes) <= capacidade_peso[j]

# Constraint 2: Each package must be allocated to at most one vehicle
for i in pacotes:
    prob += lpSum(x[i, j] for j in veiculos) <= 1

# Solve the problem
prob.solve()

# Create the final table with the results
resultados = []

for i in pacotes:
    alocado = False
    for j in veiculos:
        if x[i, j].value() == 1:
            resultados.append([i, pesos[i-1], j])  # Allocated package
            alocado = True
            break
    if not alocado:
        resultados.append([i, pesos[i-1], 'Not Allocated'])  # Package not allocated

# Create DataFrame with results
df_resultado = pd.DataFrame(resultados, columns=['Package ID', 'Weight', 'Allocated Vehicle'])

# Sort first by 'Allocated Vehicle' and then by 'Weight' in descending order
df_resultado = df_resultado.sort_values(by=['Allocated Vehicle', 'Weight'], ascending=[True, False])

# Display the final sorted DataFrame
print(df_resultado)


    ID do Pacote       Peso Veículo Alocado
52            53  49.430318     Não Alocado
20            21  48.952299     Não Alocado
72            73  48.861293     Não Alocado
70            71  48.846514     Não Alocado
8              9  48.219475     Não Alocado
..           ...        ...             ...
26            27   8.024311              V3
63            64   7.770965              V3
47            48   7.317389              V3
61            62   6.408382              V3
14            15   4.480767              V3

[100 rows x 3 columns]


In [ ]:
from google.colab import files

arquivo_excel = 'df_resultado.xlsx'
df_resultado.to_excel(arquivo_excel, index=False, engine='openpyxl')

# download
files.download(arquivo_excel)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# Calculating the total weight capacity occupied in each vehicle
capacidade_ocupada = {veiculo: 0 for veiculo in veiculos}

for i in pacotes:
    for j in veiculos:
        if x[i, j].value() == 1:
            capacidade_ocupada[j] += pesos[i-1]  # Adding the weight of the allocated package to the vehicle

# Displaying the occupied capacity in each vehicle
for veiculo, peso_total in capacidade_ocupada.items():
    print(f"Vehicle {veiculo} - Occupied Weight Capacity: {peso_total:.2f} kg")

# Checking remaining capacity
for veiculo in veiculos:
    capacidade_restante = capacidade_peso[veiculo] - capacidade_ocupada[veiculo]
    print(f"Vehicle {veiculo} - Remaining Capacity: {capacidade_restante:.2f} kg")


Veículo V1 - Capacidade de Peso Ocupada: 198.17 kg
Veículo V2 - Capacidade de Peso Ocupada: 284.38 kg
Veículo V3 - Capacidade de Peso Ocupada: 390.19 kg
Veículo V1 - Capacidade Restante: 1.83 kg
Veículo V2 - Capacidade Restante: 15.62 kg
Veículo V3 - Capacidade Restante: 9.81 kg


In [ ]:
capacidade_restante = 2416.69-(198.17 +284.38+390.19)
print(capacidade_restante)

1543.95


In [ ]:
from google.colab import files

# Create the content for README.md
readme_content = """
# Logistical Load Maximization Project: Packing Analysis

## Introduction

This project aims to enhance logistical capabilities through the maximization of package load, considering constraints related to fragility and weight of the vehicles.

## Project Structure

The project consists of a Python script utilizing the `pandas` and `numpy` libraries to perform the following stages:

1. **Data Generation**: Random generation of package data.
2. **Linear Programming**: Por meio da biblioteca PULP, foram definidas as restrições de peso e fragilidade, bem como o carregamento de cada pacote para um dos veículos.
  O problema foi construído em estrutura de Machine Learning, como resolução por maximização.
3. **Load**: Exporting the processed data.

## Usage

1. Clone this repository or download the Python script.
2. Run the script in a Python environment.

## Features

- Data is already generated by the code itself; feel free to substitute it.
- Processes and cleans the collected data.
"""

# Save the content to a README.md file
with open('README-packagenumber.md', 'w') as f:
    f.write(readme_content)

# Download the README.md file
files.download('README-packagenumber.md')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>